## Very Basics

There are much more extensive notes on DOM manipulation in my TypeScript Standard Libarary snippets [here](https://github.com/davidpet/snippets/blob/main/typescript/Standard%20Library.ipynb).

In [1]:
%%html

<script>
    (() => { // lambda because of how jupyter notebooks deal with top-level names in JS
    // Get a reference to an existing div by its id. Replace 'existing-div-id' 
    // with the actual id of the div you're interested in.
    const existingDiv = document.getElementById('existing-div-id');

    // Create a new element. This could be any valid HTML tag name.
    // In this example, we'll create a div.
    const div = document.createElement('div');

    // Get the parent of the existing div
    const parentDiv = existingDiv.parentNode;

    // Find the index of the existing div within its parent's children
    const index = Array.from(parentDiv.children).indexOf(existingDiv);

    // Set the new div's text content to include the index
    div.textContent = `Hello, world! Parent div is child number ${index} of its parent.`;

    // Or, we could give it a CSS class:
    div.className = 'my-class';

    // We can also set styles directly. Here's how to change the color:
    div.style.color = 'blue';

    // Let's create a click handler for our div
    const clickHandler = (e) => {
        console.log(e);
        e.target.textContent = "Clicked!";
        // Unregister the handler after one click
        div.removeEventListener('click', clickHandler);
    };

    // Now we register the click handler to the div
    div.addEventListener('click', clickHandler);

    // Finally, we append the new element to the existing one.
    // In this example, the div will become a child of the existing div.
    existingDiv.appendChild(div);
    })()
</script>

<body>
    <div>Div1</div>
    <div id="existing-div-id">Div2</div>
    <div>Div3</div>
</body>

## Event Phases and Propagation

__NOTE:__ Unfortunately, there's an issue with the Jupyter JS implementation that prevents the div and button events from firing, but I tested it in a normal web page.

In general, the flow of events is as follows:
- __Capture Phase__ ("event capturing")
  - Events propagate from `window` down to `document` and down the DOM tree until the lowest level target
- __Bubble Phase__
  - Events propagate back from the lowest level target all the way up to `window` again
- __Target Phase__
  - This just means that on a given target in a given phase, handlers are called in the __order they were added__
  
As shown below, events are added to the __bubble phase by default__, but there's an optional 3rd boolean parameter to specify __capture phase__ instead.

These methods of the `Event` object allow you to control the propagation when needed:
- `stopPropagation`
  - the event will fire on the remaining handlers on the current target in the current phase and then stop
  - eg. in capture phase, you can install a global handler to stop children from seeing clicks directly
- `stopImmediatePropagation`
  - like `stopPropagation` but also stops other handlers on the same level (after this one) from getting called

In [1]:
%%html

<body>
    <div>
        <button>
            Click Here
        </button>
    </div>
</body>

<script>
    (() => { // calling as lambda due to how jupyter handles rerunning cells
        // You have to look at the actual JS console (not the notebook) to see these
        document.addEventListener('click', () => console.log('Document bubble 1'));
        document.addEventListener('click', () => console.log('Document bubble 2'));
        document.addEventListener('click', () => console.log('Document capture 1'), true);
        document.addEventListener('click', () => console.log('Document capture 2'), true);
        window.addEventListener('click', () => console.log('Window bubble'));
        
        document.querySelector('div').addEventListener('click', () => console.log('Div bubble'));
        document.querySelector('div').addEventListener('click', () => console.log('Div capture'), true);
        
        document.querySelector('div button').addEventListener('click', () => console.log('Button bubble 1'));
        document.querySelector('div button').addEventListener('click', () => console.log('Button capture 1'), true);
        
        document.querySelector('div button').addEventListener('click', () => console.log('Button bubble 2'));
        document.querySelector('div button').addEventListener('click', () => console.log('Button capture 2'), true);
        
        // Expected Output:
        /*
            Document capture 1
            Document capture 2
            Div capture
            Button capture 1
            Button capture 2
            Button bubble 1
            Button bubble 2
            Div bubble
            Document bubble 1
            Document bubble 2
            Window bubble
        */
    })();
</script>